In [ ]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

In [ ]:
#Instalamos la libreria de PLOTLY
%pip install plotly

In [3]:
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
! npm install localtunnel


up to date in 3s

3 packages are looking for funding
  run `npm fund` for details


In [8]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [10]:
%%writefile app.py 
#Creamos el archivo de la APP en el interprete principal (Phyton)
#####################################################
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
######################################################
#Definimos la instancia
@st.cache_resource
######################################################
#Creamos la función de carga de datos
def load_data():
   #Lectura del archivo csv
   df=pd.read_csv("titanic_despliegue.csv")
    #Rellenamos nulos
   df =df.fillna(method="bfill")
   df =df.fillna(method="ffill")  
   Lista=['Survived', 'Pclass', 'Sex', 'Age']
   return df, Lista
###############################################################################
#Cargo los datos obtenidos de la función "load_data"
df, Lista = load_data()
###############################################################################
#CREACIÓN DEL DASHBOARD
#Generamos las páginas que utilizaremos en el diseño
##############################################################################
#Generamos los encabezados para la barra lateral (sidebar)
st.sidebar.title("TITANIC")

Overwriting app.py
